Basic MLP that employs a forward propagation then back propagation algorithm.
Works as follows:

Input layer corresponding to size of data being fed in

Hidden layer

Output layer of one node

-Initially randomise weights and set biases to -1

-Apply forward propagation algorithm using a choice of activation - linear is what is used

-Calculate difference between target variable and output

-Apply backpropagation algorithm and update weights

-Add one to epoch, save weights and start over

-At a selected Epoch mutliply weights by some factor

-At end, visualise the training with minimisation and weight matrix

In [ ]:
class MultiLayerPerceptron(BaseEstimator, ClassifierMixin): 
    def __init__(self, params=None):     
        if (params == None):
            self.inputLayer = 10                       # Input Layer
            self.hiddenLayer = 50                      # Hidden Layer
            self.OutputLayer = 1                       # Outpuy Layer
            self.learningRate = 0.001                  # Learning rate
            self.max_epochs = 50                      # Epochs
            self.BiasHiddenValue = -1                   # Bias HiddenLayer initial
            self.BiasOutputValue = -1                  # Bias OutputLayer initial
            self.activ = self.activation['linear'] # Activation function
            self.deriv = self.derivative['linear']
            self.perturn_weights = False
        else:
            self.inputLayer = params['InputLayer']
            self.hiddenLayer = params['HiddenLayer']
            self.OutputLayer = params['OutputLayer']
            self.learningRate = params['LearningRate']
            self.max_epochs = params['Epochs']
            self.BiasHiddenValue = params['BiasHiddenValue']
            self.BiasOutputValue = params['BiasOutputValue']
            self.activ = self.activation[params['ActivationFunction']]
            self.deriv = self.derivative[params['ActivationFunction']]
            self.perturn_weights = params['Epochs']
        
        'Starting Bias and Weights'
        self.WEIGHT_hidden = self.starting_weights(self.hiddenLayer, self.inputLayer)
        self.WEIGHT_output = self.starting_weights(self.OutputLayer, self.hiddenLayer)
        self.BIAS_hidden = np.array([self.BiasHiddenValue for i in range(self.hiddenLayer)])
        self.BIAS_output = np.array([self.BiasOutputValue for i in range(self.OutputLayer)])
        self.output_number = 1 #defines how many numbers are in the output
        
        
    pass
    
    def starting_weights(self, x, y):
        return [[2  * random.random() - 1 for i in range(x)] for j in range(y)]

    activation = {
         'sigmoid': (lambda x: 1/(1 + np.exp(-x))),
            'tanh': (lambda x: np.tanh(x)),
            'Relu': (lambda x: x*(x > 0)),
            'linear': (lambda x: x)
               }
    derivative = {
         'sigmoid': (lambda x: x*(1-x)),
            'tanh': (lambda x: 1-x**2),
            'Relu': (lambda x: 1 * (x>0)),
            'linear': (lambda x: np.ones_like(x))
               }
 
    def Backpropagation_Algorithm(self, inputs, ERROR_output):

        'Stage 1 - Error: OutputLayer'
        DELTA_output = ((-1)*(ERROR_output) * self.deriv(self.OUTPUT_L2))

        'Stage 2 - Update weights OutputLayer and HiddenLayer'
        for i in range(self.hiddenLayer):
            for j in range(self.OutputLayer):
                self.WEIGHT_output[i][j] -= (self.learningRate * (DELTA_output[j] * self.OUTPUT_L1[i]))
                self.BIAS_output[j] -= (self.learningRate * DELTA_output[j])
          
        'Stage 3 - Error: HiddenLayer'
        delta_hidden = np.matmul(self.WEIGHT_output, DELTA_output)* self.deriv(self.OUTPUT_L1)

        'Stage 4 - Update weights HiddenLayer and InputLayer(x)'
        for i in range(self.OutputLayer):
            for j in range(self.hiddenLayer):
                self.WEIGHT_hidden[i][j] -= (self.learningRate * (delta_hidden[j] * inputs[i]))
                self.BIAS_hidden[j] -= (self.learningRate * delta_hidden[j])
                
    def show_err_graphic(self,error,epochs):
        plt.figure(figsize=(9,4))
        plt.plot(epochs, error, "m-",color="b", marker=11)
        plt.xlabel("Number of Epochs")
        plt.ylabel("Absolute Error ");
        plt.title("Error Minimization")
        plt.show()

    def visualise_weights(self,weights):
        layer_idx = 0  # Index of the layer you want to visualize
        weights = weights[layer_idx]  # Weights matrix for the selected layer
        
        # Reshape the weights matrix to a 2D grid
        weights= np.array(weights)
        grid_weights = weights.reshape(self.hiddenLayer, self.inputLayer)  # Assuming visualising hidden layer
        
        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Plot the weights as an image
        im = ax.imshow(grid_weights, cmap='viridis')  # Choose a colormap
        
        # Add colorbar for mapping values to colors
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel('Weight Values', rotation=-90, va="bottom")
        
        # Set title and labels
        ax.set_title(f'Layer {layer_idx} Weights Visualization')
        ax.set_xlabel('Input Neuron Index')
        ax.set_ylabel('Hidden Neuron Index')
        
        # Show the plot
        plt.show()
    
    def predict(self, X, y):
        'Returns the predictions for every element of X'
        my_predictions = []
        'Forward Propagation'
        forward = np.matmul(X,self.WEIGHT_hidden) + self.BIAS_hidden
        forward = np.matmul(forward, self.WEIGHT_output) + self.BIAS_output
                                 
        for i in forward:
            my_predictions.append(i)
            
        array_score = []
        for i in range(len(my_predictions)):
            error = y[i]-my_predictions[i]
            #Check if the prediction is close to the actual
            if np.abs(error) <0.05:
                correct = 1
            else:
                correct=0
            array_score.append(correct)
            
        total_accuracy = (sum(array_score) / len(array_score))*100
        print("Total Accuracy:", total_accuracy)

        return my_predictions, total_accuracy

    def fit(self, X, y):  
        count_epoch = 1

        epoch_array = []
        error_array = []
        W0 = []
        W1 = []
        while(count_epoch <= self.max_epochs):
            for inputs, target in zip(X,y): 
                'Stage 1 - (Forward Propagation)'
                self.OUTPUT_L1 = self.activ((np.dot(inputs, self.WEIGHT_hidden) + self.BIAS_hidden.T))
                self.OUTPUT_L2 = self.activ((np.dot(self.OUTPUT_L1, self.WEIGHT_output) + self.BIAS_output.T))

                #Find the difference between the correct datapoint and the NN output
                total_error = target - self.OUTPUT_L2

                'Backpropagation : Update Weights'
                self.Backpropagation_Algorithm(inputs, total_error)
                
            if((count_epoch % 50 == 0)or(count_epoch == 1)):
                print("Epoch ", count_epoch, "- Total Error: ",total_error)

                error_array.append(total_error)
                epoch_array.append(count_epoch)
            #Perturb the weights on epoch 13 to test.
            if (count_epoch== 13) and self.perturn_weights:
                print("Perturbing weights by 1.3")
                if count_epoch == 13 and self.perturn_weights:
                    for row_index, row in enumerate(self.WEIGHT_hidden):
                        for col_index, value in enumerate(row):
                            try:
                                self.WEIGHT_hidden[row_index][col_index] *= 1.3
                            except TypeError:
                                print("Non-numeric value found in self.WEIGHT_hidden.")

            W0.append(self.WEIGHT_hidden)
            W1.append(self.WEIGHT_output)
             
            count_epoch += 1
        #Show loss minimisation
        self.show_err_graphic(error_array,epoch_array)
        
        #Show heatmap of weights
        self.visualise_weights(W0)
        """
        plt.plot(W0[0])
        plt.title('Weight Hidden update during training')
        plt.legend(['neuron1', 'neuron2', 'neuron3', 'neuron4', 'neuron5'])
        plt.ylabel('Value Weight')
        plt.show()
        """
        plt.plot(W1[0])
        plt.title('Weight Output update during training')
        plt.legend(['neuron1'])
        plt.ylabel('Value Weight')
        plt.show()

        return self

Currently generating data via both gaussian and interval spacing between 0 and 1. Target is std of gaussian and mutliplication of the step number respectively.

In [ ]:
def randomly_generate_data_gaussian(num_samples):
    y_train = []
    x_train = []
    mean =  0
    x_values = np.linspace(-3, 3, 10)
    
    for i in range(num_samples):
        #standard_deviation = random.uniform(0.5, 1.5) #Produces a random float value
        standard_deviation = random.randrange(1,5) #Produces a integer between the range
        #gaussian_vector = gaussian(x_values, mean, standard_deviation)
        rv = norm(loc = mean, scale = standard_deviation)
        gaussian_vector = rv.pdf(x_values)
        y_train.append([standard_deviation])  # Append standard_deviation as a new row
        x_train.append(list(gaussian_vector))  # Append gaussian vector as a new row
    #plt.plot(x_values,gaussian_vector)
    return np.array(x_train), np.array(y_train)
    

def randomly_generate_data_uniform(num_samples):
    y_train = []
    x_train = []
    x_values = np.linspace(0, 1, 10)
    
    for i in range(num_samples):
        #standard_deviation = random.uniform(0.5, 1.5) #Produces a random float value
        multi_num = random.randrange(1,8) #Produces a integer between the range
        numbers = multi_num*x_values

        y_train.append([multi_num])  # Append standard_deviation as a new row
        x_train.append(list(numbers))  # Append gaussian vector as a new row
    #plt.plot(x_values,gaussian_vector)
    return np.array(x_train), np.array(y_train)

Generate data and split

In [ ]:
random.seed(471)  
x_train, y_train = randomly_generate_data_uniform(1000)
x_test, y_test = randomly_generate_data_uniform(100)
#%%
Perceptron = MultiLayerPerceptron()
Perceptron.fit(x_train,y_train)
#%%
predictions, accuracy = Perceptron.predict(x_test, y_test)
